In [ ]:
# 导入必要的库
import pandas as pd              # 数据处理，核心函数：pd.read_csv(), DataFrame操作
import numpy as np               # 数值计算，支持多维数组和数学运算
from sklearn.model_selection import (
    train_test_split,            # 数据集划分函数，参数：test_size, random_state
    cross_val_score              # 交叉验证得分计算（本代码未直接使用）
)
from sklearn.linear_model import (
    LinearRegression,            # 线性回归模型（最小二乘法）
    Ridge,                       # 岭回归（L2正则化），参数：alpha（正则化强度）
    Lasso                        # Lasso回归（L1正则化），参数：alpha（正则化强度）
)
from sklearn.ensemble import RandomForestRegressor  # 随机森林回归，参数：n_estimators（树的数量）
from sklearn.preprocessing import StandardScaler    # 标准化（均值0，标准差1）
from sklearn.metrics import (
    mean_squared_error,          # 均方误差（MSE），越小越好
    r2_score                     # R²分数，越接近1越好
)
import matplotlib.pyplot as plt  # 绘图库，用于可视化结果

# 可选导入XGBoost和LightGBM（如果未安装则跳过）
try:
    from xgboost import XGBRegressor  # XGBoost回归模型，参数：n_estimators, verbosity（日志输出）
    xgb_installed = True
except ImportError:
    xgb_installed = False        # 标记XGBoost是否可用
try:
    from lightgbm import LGBMRegressor  # LightGBM回归模型，参数：n_estimators
    lgb_installed = True
except ImportError:
    lgb_installed = False        # 标记LightGBM是否可用

# --------------------------
# 工具函数定义
# --------------------------
def ensure_id_column(df, start_id=1501):
    """
    确保数据框包含'id'列，若无则自动生成从start_id开始的连续ID
    参数:
        df (pd.DataFrame): 输入数据框
        start_id (int): 起始ID值，默认为1501
    返回:
        pd.DataFrame: 包含'id'列的数据框
    """
    if 'id' not in df.columns:
        df['id'] = range(start_id, start_id + len(df))  # 生成连续ID
    return df

def feature_engineering(df, is_train=True):
    """
    特征工程函数，处理日期字段并生成统计特征
    参数:
        df (pd.DataFrame): 输入数据框
        is_train (bool): 是否为训练集（True则生成标签，False则跳过）
    返回:
        X (pd.DataFrame): 特征矩阵
        y (pd.Series): 标签列（仅当is_train=True时返回）
    """
    df = df.copy()
    # 日期字段处理（如果存在'date'列）
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])  # 转为datetime类型
        df['year'] = df['date'].dt.year          # 提取年份
        df['month'] = df['date'].dt.month        # 提取月份
        df['day'] = df['date'].dt.day            # 提取日
        df['day_of_week'] = df['date'].dt.dayofweek  # 提取星期几（0=周一，6=周日）
        df = df.drop(columns='date')             # 删除原始日期列
    
    # 统计特征生成（仅训练集计算滚动特征，测试集填0保持特征一致性）
    if is_train and 'close' in df.columns:
        # 3日收盘价滚动均值和标准差（min_periods=1允许窗口不满时计算）
        df['close_rolling_mean3'] = df['close'].rolling(window=3, min_periods=1).mean()
        df['close_rolling_std3'] = df['close'].rolling(window=3, min_periods=1).std().fillna(0)
    else:
        # 测试集填充同名特征（避免特征维度不一致）
        df['close_rolling_mean3'] = 0
        df['close_rolling_std3'] = 0
    
    # 分离特征和标签
    if is_train:
        X = df.drop(columns=['id', 'close'], errors='ignore')  # 删除ID和标签列（errors='ignore'防止列不存在时报错）
        y = df['close']
    else:
        X = df.drop(columns=['id'], errors='ignore')
        y = None
    return X, y

# --------------------------
# 主函数
# --------------------------
def main():
    # 1. 数据读取
    train = pd.read_csv('/kaggle/input/houkongtest/stock_train_data.csv')  # 读取训练集
    test = pd.read_csv('/kaggle/input/houkongtest/stock_test_data.csv')    # 读取测试集
    test = ensure_id_column(test, start_id=1501)  # 确保测试集有'id'列

    # 2. 特征工程
    X, y = feature_engineering(train, is_train=True)   # 处理训练集，生成特征和标签
    X_test, _ = feature_engineering(test, is_train=False)  # 处理测试集，仅生成特征
    id_test = test['id']  # 保存测试集ID用于最终结果

    # 3. 划分验证集
    # train_test_split参数：
    # - test_size=0.2: 验证集占比20%
    # - random_state=42: 固定随机种子保证可复现
    X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # 4. 标准化处理
    scaler = StandardScaler()               # 初始化标准化器
    X_tr_scaled = scaler.fit_transform(X_tr)  # 在训练集上拟合标准化参数并转换
    X_val_scaled = scaler.transform(X_val)  # 验证集转换（使用训练集的参数）
    X_test_scaled = scaler.transform(X_test)  # 测试集转换

    # 5. 多模型对比
    models = [
        ('LinearRegression', LinearRegression()),  # 线性回归（无正则化）
        ('Ridge', Ridge(alpha=1.0)),                # 岭回归（alpha=1.0）
        ('Lasso', Lasso(alpha=0.1)),                # Lasso回归（alpha=0.1）
        ('RandomForest', RandomForestRegressor(n_estimators=200, random_state=42)),  # 随机森林（200棵树）
    ]
    # 如果安装了XGBoost和LightGBM则添加到模型列表
    if xgb_installed:
        models.append(('XGBoost', XGBRegressor(n_estimators=200, random_state=42, verbosity=0)))  # verbosity=0关闭训练日志
    if lgb_installed:
        models.append(('LightGBM', LGBMRegressor(n_estimators=200, random_state=42)))

    # 模型训练与评估
    best_score = -np.inf  # 初始化最佳R²为负无穷
    best_model = None     # 保存最优模型对象
    best_name = ""        # 保存最优模型名称
    results = []          # 保存所有模型结果

    for name, model in models:
        model.fit(X_tr_scaled, y_tr)                   # 训练模型
        val_pred = model.predict(X_val_scaled)         # 预测验证集
        r2 = r2_score(y_val, val_pred)                # 计算R²
        mse = mean_squared_error(y_val, val_pred)      # 计算MSE
        results.append((name, r2, mse))                # 记录结果
        print(f"{name} 验证集R2: {r2:.4f}, MSE: {mse:.4f}")  # 打印结果
        if r2 > best_score:                            # 更新最优模型
            best_score = r2
            best_model = model
            best_name = name

    print(f"\n最优模型：{best_name}，验证集R2: {best_score:.4f}")  # 输出最优模型

    # 6. 可视化最优模型效果
    val_pred = best_model.predict(X_val_scaled)        # 最优模型预测验证集
    plt.figure(figsize=(8,6))                          # 设置画布大小
    plt.scatter(y_val, val_pred, alpha=0.6)            # 绘制真实值vs预测值散点图
    plt.xlabel('真实值')                                # x轴标签
    plt.ylabel('预测值')                                # y轴标签
    plt.title(f'最优模型({best_name})验证集预测效果')     # 标题
    plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--')  # 绘制红色对角线（理想情况）
    plt.show()                                          # 显示图表

    # 7. 预测测试集并保存结果
    test_pred = best_model.predict(X_test_scaled)       # 预测测试集
    res_df = pd.DataFrame({'id': id_test, 'close': test_pred})  # 创建结果数据框
    res_df.to_csv('/kaggle/working/ccf.csv', index=False)       # 保存为CSV（不保留索引）
    print("最优模型预测结果已保存至 /kaggle/working/ccf.csv")    # 输出提示

# 程序入口
if __name__ == "__main__":
    main()